In [2]:
import json
# read json file
with open('wasmbench_index_distribution.json') as json_file:
    data = json.load(json_file)
len(data)

8303

In [3]:
from collections import Counter

index_exprs = Counter()
for binary, x in data.items():
    for expr, cnt in x['index_distribution'].items(): 
        index_exprs[expr] += cnt['count']
print(index_exprs.most_common(10))

[('i32.add(i32.and(<local>, i32.const <const>), i32.const <const>)', 696411), ('i32.add(i32.and(i32.load(i32.add(i32.load(<local>), i32.const <const>)), i32.const <const>), i32.const <const>)', 476095), ('i32.add(i32.and(i32.load(i32.add(i32.load(<param>), i32.const <const>)), i32.const <const>), i32.const <const>)', 448088), ('i32.add(i32.and(<param>, i32.const <const>), i32.const <const>)', 414956), ('i32.load(i32.add(i32.load(<param>), i32.const <const>))', 209146), ('i32.add(i32.and(i32.load(<local>), i32.const <const>), i32.const <const>)', 191373), ('i32.add(i32.and(i32.load(<param>), i32.const <const>), i32.const <const>)', 182918), ('<local>', 172279), ('i32.load(i32.add(i32.load(<local>), i32.const <const>))', 170414), ('<param>', 144970)]


In [4]:
accounts_for = 0
total = sum(index_exprs.values())
print('total:', total)
for expr, cnt in index_exprs.most_common():
    accounts_for += float(cnt)/float(total)
    # if accounts_for > 0.999:
    #     break
    print(f'{cnt:10} {expr}')

total: 4203474
    696411 i32.add(i32.and(<local>, i32.const <const>), i32.const <const>)
    476095 i32.add(i32.and(i32.load(i32.add(i32.load(<local>), i32.const <const>)), i32.const <const>), i32.const <const>)
    448088 i32.add(i32.and(i32.load(i32.add(i32.load(<param>), i32.const <const>)), i32.const <const>), i32.const <const>)
    414956 i32.add(i32.and(<param>, i32.const <const>), i32.const <const>)
    209146 i32.load(i32.add(i32.load(<param>), i32.const <const>))
    191373 i32.add(i32.and(i32.load(<local>), i32.const <const>), i32.const <const>)
    182918 i32.add(i32.and(i32.load(<param>), i32.const <const>), i32.const <const>)
    172279 <local>
    170414 i32.load(i32.add(i32.load(<local>), i32.const <const>))
    144970 <param>
    107066 i32.load(<param>)
     96734 i32.load(i32.const <const>)
     96265 i32.load(i32.add(<param>, i32.const <const>))
     89983 i32.load(i32.add(i32.load(i32.add(<param>, i32.const <const>)), i32.const <const>))
     83558 i32.add(i32.and(

In [5]:
import re

instrs = Counter()
for expr, cnt in index_exprs.items():
    expr = re.sub('\[[if3264, ]*\] -> \[[if3264, ]*\]', '', expr)
    expr = re.sub('call [^ ]+', 'call', expr)
    for instr in re.split(r'\(|\)| |,', expr):
        if instr != '' and instr != '<const>':
            instrs[instr] += cnt

accounts_for = 0
total = sum(instrs.values())
print('total:', total)
for instr, cnt in instrs.most_common():
    accounts_for += float(cnt)/float(total)
    if accounts_for > 0.999:
        break
    print(f'{cnt:10} {cnt/total:%} {instr}')

total: 24606068
   8002108 32.520872% i32.const
   4955565 20.139605% i32.add
   4403184 17.894708% i32.load
   2856630 11.609453% i32.and
   2224822 9.041762% <local>
   1905891 7.745614% <param>
     93447 0.379772% i32.shl
     41921 0.170369% <stack>
     30854 0.125392% i32.sub
     27234 0.110680% i32.shr_u
     26721 0.108595% i32.wrap_i64
     12734 0.051751% call
